In [338]:
import pandas as pd
import numpy as np
import os
from io import StringIO
import re
import csv

import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import graphviz

from sklearn import preprocessing
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

import pydotplus

%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 10]

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 400)

os.getcwd()



'/Users/lubagloukhov/Documents/Consulting/PiqueSolutions/JavaSE_202007/scripts'

In [339]:
raw_df = pd.read_csv('../data/input/SurveyReport-rawdata_20200702_deduped.csv')  

In [340]:
raw_df.shape

(228, 162)

# 1. Create: <br> &emsp; - short_name for columns

In [341]:
[x for x in raw_df.columns]

['Response ID',
 'IP Address',
 'Timestamp (mm/dd/yyyy)',
 'Duplicate',
 'Time Taken to Complete (Seconds)',
 'Seq. Number',
 'External Reference',
 'Custom Variable 1',
 'Custom Variable 2',
 'Custom Variable 3',
 'Custom Variable 4',
 'Custom Variable 5',
 'Respondent Email',
 'Email List',
 'Country Code',
 'Region',
 'Pique Solutions, an independent analyst firm, is conducting research sponsored by Oracle among selected IT professionals at leading organizations. The objective of this survey is to gather your feedback related to your Oracle Java SE Subscription experience in order to improve the service based on your feedback.Your input will be shared with Oracle in an aggregate format, and neither your name nor your company’s name will be associated with your comments without your explicit consent. We would like to talk with you about the benefits and shortcomings of your Oracle Java SE Subscription, and the considerations behind your selection of Oracle Java SE.First, please respo

In [342]:
short_names = ['Response_ID',
 'IP_Address',
 'Timestamp',
 'Duplicate',
 'Time_Complete_s',
 'Seq._Number',
 'External_Reference',
 'Custom_Variable_1',
 'Custom_Variable_2',
 'Custom_Variable_3',
 'Custom_Variable_4',
 'Custom_Variable_5',
 'Respondent_Email',
 'Email_List',
 'Country_Code',
 'Region',
 'Survey_Pre_Qual',
 'Job_Title',
 'Job_Title_Other',
 'Company_Industry',
 'Company_Industry_Other]',
 'Company_Revenue',
 'OracleJavaSESubscription_Org_Duration',
 'Java_App_Org_Deploy_2020',
 'OracleJavaSESubscription_Ind_Role',
 'OracleJavaSESubscription_Ind_Role_Other',
 'Survey_Consent',
 'Government_Org_Ind_Employee',
 'Gift_Card_Confirm',              
 'Company_Name',
 'Thank_You',
 'Title',
 'Country',
 'State',
 'App_Dev_Outsource',
 'Java_Dev_Org',
 "Java_Dev_Dist_Org",
 "Java_Dev_Dist_JVM",
 "Java_Dev_Dist_Python",
 "Java_Dev_Dist_JavaScript",
 "Platform_Dev_Org_Primary",
 "Platform_Dev_Org_Comment",
 "Platform_Dev_Org_LRHEL_P",
 "Platform_Dev_Org_LOracle_P",
 "Platform_Dev_Org_LUbuntu_P",
 "Platform_Dev_Org_LSUSE_P",
 "Platform_Dev_Org_LinuxOther_P",
 "Platform_Dev_Org_MacOS_P",
 "Platform_Dev_Org_Solaris_P",
 "Platform_Dev_Org_Windows_P",
 "Platform_Dev_Org_LinuxOther_Comment",
 'Use_Org_EclipseIDE',
 'Sat_Org_EclipseIDE',
 'Use_Org_IntelliJIDE',
 'Sat_Org_IntelliJIDE',
 'Use_Org_JDeveloper',
 'Sat_Org_JDeveloper',
 'Use_Org_NetBeansIDE',
 'Sat_Org_NetBeansIDE',
 'Use_Org_RationalAppDev',
 'Sat_Org_RationalAppDev',
 'Use_Org_VisualStudio',
 'Sat_Org_VisualStudio',
 'Use_Org_Other',
 'Sat_Org_Other',
 'Use_Sat_Org_Other_Comment',
 'Use_Org_JavaSEAMC',
 'Sat_Org_JavaSEAMC',
 'Use_Org_JavaSEAMC_UsageLog',
 'Sat_Org_JavaSEAMC_UsageLog',
 'Use_Org_JavaSEDepRuleSet',
 'Sat_Org_JavaSEDepRuleSet',
 'Use_Org_JavaSEMSICustTools',
 'Sat_Org_JavaSEMSICustTools',
 'Use_Org_JavaSEFlightRec',
 'Sat_Org_JavaSEFlightRec',
 'JavaSESub_PrePurchasePeriod',             
 'Information_Channel_Imp_OWebsites',             
 'Information_Channel_Imp_OSalesPros',             
 'Information_Channel_Imp_OMarketing',             
 'Information_Channel_Imp_Blogs',             
 'Information_Channel_Imp_ExtConsultants',             
 'Information_Channel_Imp_SocialMedia',
 'Information_Channel_Imp_UserComm',             
 'Information_Channel_Imp_IndustryPubs',             
 'Information_Channel_Imp_PeerRec',             
 'Information_Channel_Imp_Other',             
 'Information_Channel_Imp_Other_Comment',
 'OJavaSESubscription_Elem_Imp_Org_ContinuedInvest',
 'OJavaSESubscription_Elem_Sat_Org_ContinuedInvest',
 'OJavaSESubscription_Elem_Imp_Org_PatchesOlderV',
 'OJavaSESubscription_Elem_Sat_Org_PatchesOlderV',
 'OJavaSESubscription_Elem_Imp_Org_JavaWebStartUpdates',
 'OJavaSESubscription_Elem_Sat_Org_JavaWebStartUpdates',
 'OJavaSESubscription_Elem_Imp_Org_DesktopManageUpdates',
 'OJavaSESubscription_Elem_Sat_Org_DesktopManageUpdates',
 'OJavaSESubscription_Elem_Imp_Org_AutoUpdateTools',
 'OJavaSESubscription_Elem_Sat_Org_AutoUpdateTools',
 'OJavaSESubscription_Elem_Imp_Org_Monitoring',
 'OJavaSESubscription_Elem_Sat_Org_Monitoring',
 'OJavaSESubscription_Elem_Imp_Org_MyOSupport',
 'OJavaSESubscription_Elem_Sat_Org_MyOSupport',
 'OJavaSESubscription_Elem_Imp_Org_AccessCloud',
 'OJavaSESubscription_Elem_Sat_Org_AccessCloud',
 'OJavaSESubscription_Elem_Imp_Org_FlexLicensing',
 'OJavaSESubscription_Elem_Sat_Org_FlexLicensing',
 'OJavaSESubscription_Elem_Imp_Org_LegacyJava',
 'OJavaSESubscription_Elem_Sat_Org_LegacyJava',
 'OJavaSESubscription_Elem_Imp_Org_Other',
 'OJavaSESubscription_Elem_Sat_Org_Other',
 'OJavaSESubscription_Elem_Imp_Sat_Org_Other_Comment',
               
 'OJavaSESubscription_Attr_Imp_CommMethods',
 'OJavaSESubscription_Attr_Sat_CommMethods',
 'OJavaSESubscription_Attr_Imp_PhoneSupport',
 'OJavaSESubscription_Attr_Sat_PhoneSupport',
 'OJavaSESubscription_Attr_Imp_SSKnowledge',
 'OJavaSESubscription_Attr_Sat_SSKnowledge',
 'OJavaSESubscription_Attr_Imp_CoNeedsUnd',
 'OJavaSESubscription_Attr_Sat_CoNeedsUnd',
 'OJavaSESubscription_Attr_Imp_TrainingQual',
 'OJavaSESubscription_Attr_Sat_TrainingQual',
 'OJavaSESubscription_Attr_Imp_RepsonseTime',
 'OJavaSESubscription_Attr_Sat_RepsonseTime',
 
 'OJavaSESubscription_OrgPurch_Imp_StaffExp',
 'OJavaSESubscription_OrgPurch_Imp_PrPerRatio',
 'OJavaSESubscription_OrgPurch_Imp_TotalCost',
 'OJavaSESubscription_OrgPurch_Imp_LargeEcosystem',
 'OJavaSESubscription_OrgPurch_Imp_VendorSupport',
 'OJavaSESubscription_OrgPurch_Imp_InnovationPotential',
 'OJavaSESubscription_OrgPurch_Imp_LowerRisk',
 'OJavaSESubscription_OrgPurch_Imp_RegulatoryCompliance',
 'OJavaSESubscription_OrgPurch_Imp_MarketTrends',
 'OJavaSESubscription_OrgPurch_Imp_Competition',
 'OJavaSESubscription_OrgPurch_Imp_HigherAgility',
 'OJavaSESubscription_OrgPurch_Imp_StrategicImperative',
 'OJavaSESubscription_OrgPurch_Imp_ThoughtLeadership',
 'OJavaSESubscription_OrgPurch_Imp_CompanyPolitics',
 'OJavaSESubscription_OrgPurch_Imp_InnovationContribution',
 'OJavaSESubscription_OrgPurch_Imp_Other',
 'OJavaSESubscription_OrgPurch_Imp_Other_comment',
 'OJavaSESubscription_Alts',
 'OJavaSESubscription_Alts_Top3_MaintenanceTools',
 'OJavaSESubscription_Alts_Top3_UpdatesTools',
 'OJavaSESubscription_Alts_Top3_CostManagement ',
 'OJavaSESubscription_Alts_Top3_Flexibility',
 'OJavaSESubscription_Alts_Top3_Familiarity',
 'OJavaSESubscription_Alts_Top3_EasierUse]',
 'OJavaSESubscription_Alts_Top3_OpenSourceTools',
 'OJavaSESubscription_Alts_Top3_Support',
 'OJavaSESubscription_Alts_Top3_AppSecurity',
 'OJavaSESubscription_Alts_Top3_DevResources',
 'OJavaSESubscription_Alts_Top3_RiskReg',
 'OJavaSESubscription_Alts_Top3_RiskAppDowntime',
 'OJavaSESubscription_Alts_Top3_Other',
 'ThanksPhone1',
 'ThanksPhone2',
 'First Name',
 'Last Name',
 'Phone',
 'Email',
 'GiftCard_Select',
 'GiftCard_Email']

In [343]:
len(short_names)

162

In [344]:
short_names[21]

'Company_Revenue'

In [345]:
len(raw_df.columns)

162

In [348]:
map_df = pd.DataFrame({'new':short_names,
             'org':raw_df.columns})
map_df.head()

,new,org
0,Response_ID,Response ID
1,IP_Address,IP Address
2,Timestamp,Timestamp (mm/dd/yyyy)
3,Duplicate,Duplicate
4,Time_Complete_s,Time Taken to Complete (Seconds)


# 2. Convert: <br> &emsp; - continuous variables to int/float, <br> &emsp; - categorical to dummy

In [350]:
col = ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Other]'

In [351]:
print(raw_df[col].isna().sum())
raw_df[col].head()

226


0    Cost of SE
1           NaN
2           NaN
3           NaN
4           NaN
Name:  From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Other], dtype: object

In [352]:
raw_df[col].value_counts()

Cost of SE                                                    1
I choose Java because it is best and use and all the above    1
Name:  From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Other], dtype: int64

In [353]:
cont_list = ['Approximately how many Java developers are in your organization?\xa0',
             "What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on Java",
             "What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on JVM languages besides Java",
 "What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms       % developing on Python",
 "What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms   % developing on JavaScript",
 
 'Please provide approximate percentage(s) for each of the development platforms listed below.  Linux (RHEL)',
 'Please provide approximate percentage(s) for each of the development platforms listed below.  Linux (Oracle)',
 'Please provide approximate percentage(s) for each of the development platforms listed below.  Linux Ubuntu',
 'Please provide approximate percentage(s) for each of the development platforms listed below.  Linux (SUSE)',
 'Please provide approximate percentage(s) for each of the development platforms listed below.  Linux (Other) *',
 'Please provide approximate percentage(s) for each of the development platforms listed below.  MacOS',
 'Please provide approximate percentage(s) for each of the development platforms listed below.  Solaris',
 'Please provide approximate percentage(s) for each of the development platforms listed below.  Windows',

# What is your organization’s level of use and level of satisfaction with each of these development tools? 
 'Eclipse IDE Level of use (0-10)\xa0[ , ]',
 'Eclipse IDE Level of satisfaction (0-10)[ , ]',
 'IntelliJ IDE Level of use (0-10)\xa0[ , ]',
 'IntelliJ IDE Level of satisfaction (0-10)[ , ]',
 'JDeveloper Level of use (0-10)\xa0[ , ]',
 'JDeveloper Level of satisfaction (0-10)[ , ]',
 'NetBeans IDE Level of use (0-10)\xa0[ , ]',
 'NetBeans IDE Level of satisfaction (0-10)[ , ]',
 'Rational Application Developer Level of use (0-10)\xa0[ , ]',
 'Rational Application Developer Level of satisfaction (0-10)[ , ]',
 'Visual Studio Level of use (0-10)\xa0[ , ]',
 'Visual Studio Level of satisfaction (0-10)[ , ]',
 'Other * Level of use (0-10)\xa0[ , ]',
 'Other * Level of satisfaction (0-10)[ , ]',  

# Please rate the level of usage of the following tools in your organization on a scale from 0 to 10 with 
# (0 = No Usage, 10 = Extremely High Usage). Also, please rate how satisfied are you with the following tools your 
#  organization uses to address these support issues. 
 'Oracle Java SE Advanced Management Console (AMC) Usage (0-10)[\xa0,\xa0]',
 'Oracle Java SE Advanced Management Console (AMC) Satisfaction (0-10)[\xa0,\xa0]',
 'Java SE Usage Logging in AMC Usage (0-10)[\xa0,\xa0]',
 'Java SE Usage Logging in AMC Satisfaction (0-10)[\xa0,\xa0]',
 'The Java SE Deployment Rule Set Usage (0-10)[\xa0,\xa0]',
 'The Java SE Deployment Rule Set Satisfaction (0-10)[\xa0,\xa0]',
 'Java SE MSI Customizations Tools  Usage (0-10)[\xa0,\xa0]',
 'Java SE MSI Customizations Tools  Satisfaction (0-10)[\xa0,\xa0]',
 'Java SE Flight Recorder Usage (0-10)[\xa0,\xa0]',
 'Java SE Flight Recorder Satisfaction (0-10)[\xa0,\xa0]',

# On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the 
# importance of the information channels that increased your awareness of Oracle Java SE Subscription.
  'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Oracle websites',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Oracle Sales professionals',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Oracle marketing materials',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Blogs',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  External consultants',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Social Media outlets (e.g. Twitter, LinkedIn, Facebook, etc.)',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  User communities',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Industry publications',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Peers’ recommendations',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Other *',

# Please rank the importance of the following elements of Oracle Java SE Subscription to your organization. 
 'Oracle’s continued investment in the Java Ecosystem and Technology Importance (0-10)[ , ]',
 'Oracle’s continued investment in the Java Ecosystem and Technology Satisfaction (0-10)[ , ]',
 'Patches and updates for older versions of Java Importance (0-10)[ , ]',
 'Patches and updates for older versions of Java Satisfaction (0-10)[ , ]',
 'Tools and updates for desktop management specifically using Java Web Start Importance (0-10)[ , ]',
 'Tools and updates for desktop management specifically using Java Web Start Satisfaction (0-10)[ , ]',
 'Tools and updates for desktop management independently of Java Web Start / Browsers Importance (0-10)[ , ]',
 'Tools and updates for desktop management independently of Java Web Start / Browsers Satisfaction (0-10)[ , ]',
 'Access to tools for automated updates and upgrade, desktop deployment rulesets and tools for Java version management  Importance (0-10)[ , ]',
 'Access to tools for automated updates and upgrade, desktop deployment rulesets and tools for Java version management  Satisfaction (0-10)[ , ]',            
 'Monitoring tools Importance (0-10)[ , ]',
 'Monitoring tools Satisfaction (0-10)[ , ]',
 'MOS (My Oracle Support) Importance (0-10)[ , ]',
 'MOS (My Oracle Support) Satisfaction (0-10)[ , ]',
 'Access Cloud Workload and On-premise Importance (0-10)[ , ]',
 'Access Cloud Workload and On-premise Satisfaction (0-10)[ , ]',
 'Flexible licensing terms  Importance (0-10)[ , ]',
 'Flexible licensing terms  Satisfaction (0-10)[ , ]',
 'Legacy Java application support Importance (0-10)[ , ]',
 'Legacy Java application support Satisfaction (0-10)[ , ]',
 'Other * Importance (0-10)[ , ]',
 'Other * Satisfaction (0-10)[ , ]',

# Please rate the importance of the following attributes pertaining to your Java SE Subscription. 
 'Methods of communication Importance (0-10)[\xa0,\xa0]',
 'Methods of communication Satisfaction (0-10)[\xa0,\xa0]',
 'Quality of phone support Importance (0-10)[\xa0,\xa0]',
 'Quality of phone support Satisfaction (0-10)[\xa0,\xa0]',
 'Knowledge of support staff Importance (0-10)[\xa0,\xa0]',
 'Knowledge of support staff Satisfaction (0-10)[\xa0,\xa0]',
 'Oracles understanding of your companys needs Importance (0-10)[\xa0,\xa0]',
 'Oracles understanding of your companys needs Satisfaction (0-10)[\xa0,\xa0]',
 'Quality of training provided to your staff Importance (0-10)[\xa0,\xa0]',
 'Quality of training provided to your staff Satisfaction (0-10)[\xa0,\xa0]',
 'Response time to support requests Importance (0-10)[\xa0,\xa0]',
 'Response time to support requests Satisfaction (0-10)[\xa0,\xa0]',  

# Please rate the importance of the following reasons driving your organization’s decision to purchase 
# Oracle Java SE Subscription. 
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Staff expertise and availability of skills',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Price/Performance Ratio',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Total cost of ownership',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Large ecosystem of partners ',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Vendor support',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Innovation potential',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Lower risk',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Regulatory compliance',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Market trends',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Competition ',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Higher agility',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Strategic imperative',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Thought leadership and innovation',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Company politics',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Ability to contribute to ongoing innovation of Java platform',
 'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Other *'
            ]

catg_list = ['Country Code',
             'Which one of the below best describes your company’s industry grouping? ',
            'What is your job title? ',
            'What is the approximate annual revenue of your company? ',
            'How long has your organization been using Oracle Java SE Subscription for managing your Java applications? ',
            'How many Java applications does your organization plan to build or deploy in 2020? ',
            'Which of the following statements most closely describes your role in the selection and use of Oracle Java SE Subscription? (Select one)',
            'Are you an employee or official of a government organization (e.g., federal province, state, locality, public higher education institution, etc.), a government owned or controlled entity (e.g., government-owned utilities, etc.), a public international organization, or a political party or candidate for political office?',
            'Do you outsource a significant portion of your organization’s application development function? ',
            'Which platform does your organization primarily develop for?',
             'Approximately how long was the period between the time you first became aware of Java SE Subscription to the time that you purchased it? '
            ]

# Already one-hot-encoded, just convert NA to 0:
dummy0_list = [ ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better tools for maintenance and patching]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better tools for automated updates and upgrades ]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [More cost-effective management and monitoring of Java applications]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Greater flexibility]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Familiarity with platform and development tools]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Easier to use]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Access to open source tools]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better support provided by vendor, including legacy Java support]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Greater application security]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Access to skilled development resources]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Lower risk of regulatory non-compliance]',
 ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Lower risk of application downtime]'
         ]
exlc_list = ['Response ID',
 'IP Address',
 'Timestamp (mm/dd/yyyy)',
 'Duplicate',
 'Time Taken to Complete (Seconds)',
 'Seq. Number',
 'External Reference',
 'Custom Variable 1',
 'Custom Variable 2',
 'Custom Variable 3',
 'Custom Variable 4',
 'Custom Variable 5',
             'Respondent Email',
            'Email List',
            'Region',
            'Pique Solutions, an independent analyst firm, is conducting research sponsored by Oracle among selected IT professionals at leading organizations. The objective of this survey is to gather your feedback related to your Oracle Java SE Subscription experience in order to improve the service based on your feedback.Your input will be shared with Oracle in an aggregate format, and neither your name nor your company’s name will be associated with your comments without your explicit consent. We would like to talk with you about the benefits and shortcomings of your Oracle Java SE Subscription, and the considerations behind your selection of Oracle Java SE.First, please respond to a few questions to determine if you qualify:',
            'What is your job title? [Other (Please specify)]',
             'Which one of the below best describes your company’s industry grouping? [Other]',
            'Which of the following statements most closely describes your role in the selection and use of Oracle Java SE Subscription? (Select one) [Other]',
            'Would you be interested in providing input into the Java SE Subscription offering by participating in an online study about the benefits of Oracle Java SE Subscription to your organization and about your satisfaction with the Subscription?\xa0 ',
            '  You confirm that you may accept the gift card as a thank you for completing the survey and that acceptance is: (1) permissible under your employer’s internal policies; (2) permissible under the laws of your home country and any other law relevant to your employer; and (3) known to and approved by your employer’s management.\xa0Oracle reserves the right to limit attendance accordingly and pursuant to Oracle policy. \xa0 \xa0',
            'Company Name',
            'Thank you for your willingness to participate in this study.',
            'Title',
            'Country',
             'State',
             'Which platform does your organization primarily develop for? [Dynamic Comment]',
             'Please rate the importance of the following reasons driving your organization’s decision to purchase Oracle Java SE Subscription. Please rate each item on a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”.  Other * [Can you please specify?]',
            'Can you please specify Linux (Other)?',
             ' From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Other]',
 'On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription.  Other * [Can you please specify?]',
             'Can you please specify?',
            'Can you please specify?.1',
            'Alternatives to Oracle Java SE Subscription',
            'Thank you very much for your participation! We appreciate your time and input. We would love to learn more about your Java development support needs and objectives and your experience with Oracle Java SE.\xa0 Please let us know if you would be interested in a followup phone discussion. As a token of appreciation for your spending 20-30 minutes with us on the phone, we would like to offer you a $100 gift card of your choice.',
 'Thank you very much for your participation! We appreciate your time and input. We would love to learn more about your Java development support needs and objectives and your experience with Oracle Java SE.\xa0 Please let us know if you would be interested in a followup phone discussion.\xa0',
 'First Name',
 'Last Name',
 'Phone',
 'Email Address',
 'Please select a gift card you wish to receive for participation in the survey.',
 'Please provide your email address so that we can send you an electronic gift card. Please use the email address to which you received the survey invitation.\xa0']


In [354]:
print(len(cont_list))
print(len(catg_list))
print(len(dummy0_list))
print(len(exlc_list))
print(len(cont_list)+len(catg_list)+len(dummy0_list)+len(exlc_list))

97
11
12
42
162


In [355]:
print(len(raw_df.columns))

162


## convert variable types

### For cont_list vars: <br> &emsp; - remove % signs before converting to numeric <br>&emsp;  - Parse '...(0-10)...': 12>NA, x=x-1 <br> &emsp; - Parse '... a scale from 0 to 10...', x=x-1

In [385]:
mod_df = raw_df.copy()

In [386]:
print(mod_df.shape)

(228, 162)


In [387]:
temp = mod_df.filter(regex='percentage|distribution').apply(
    lambda x: x.str.extract('(\d+)', expand=False))
temp = temp.apply(pd.to_numeric, errors='coerce')
mod_df.update(temp)

In [388]:
temp = mod_df.filter(regex='(0-10)').replace([12], np.nan).apply(lambda x: x - 1)
temp = temp.apply(pd.to_numeric, errors='coerce')
mod_df[temp.columns] = temp

In [390]:
temp = mod_df[cont_list].filter(regex='a scale from 0 to 10').apply(lambda x: x - 1)
temp = temp.apply(pd.to_numeric, errors='coerce')
mod_df[temp.columns] = temp

In [391]:
mod_df[cont_list] = mod_df[cont_list].apply(pd.to_numeric, errors='coerce')

In [392]:
print(mod_df.shape)

(228, 162)


In [395]:
np.sum(mod_df[cont_list].dtypes!='float64')

0

In [396]:
mod_df[cont_list].describe()

,Approximately how many Java developers are in your organization?,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on Java,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on JVM languages besides Java,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on Python,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on JavaScript,Please provide approximate percentage(s) for each of the development platforms listed below. Linux (RHEL),Please provide approximate percentage(s) for each of the development platforms listed below. Linux (Oracle),Please provide approximate percentage(s) for each of the development platforms listed below. Linux Ubuntu,Please provide approximate percentage(s) for each of the development platforms listed below. Linux (SUSE),Please provide approximate percentage(s) for each of the development platforms listed below. Linux (Other) *,Please provide approximate percentage(s) for each of the development platforms listed below. MacOS,Please provide approximate percentage(s) for each of the development platforms listed below. Solaris,Please provide approximate percentage(s) for each of the development platforms listed below. Windows,"Eclipse IDE Level of use (0-10) [ , ]","Eclipse IDE Level of satisfaction (0-10)[ , ]","IntelliJ IDE Level of use (0-10) [ , ]","IntelliJ IDE Level of satisfaction (0-10)[ , ]","JDeveloper Level of use (0-10) [ , ]","JDeveloper Level of satisfaction (0-10)[ , ]","NetBeans IDE Level of use (0-10) [ , ]","NetBeans IDE Level of satisfaction (0-10)[ , ]","Rational Application Developer Level of use (0-10) [ , ]","Rational Application Developer Level of satisfaction (0-10)[ , ]","Visual Studio Level of use (0-10) [ , ]","Visual Studio Level of satisfaction (0-10)[ , ]","Other * Level of use (0-10) [ , ]","Other * Level of satisfaction (0-10)[ , ]","Oracle Java SE Advanced Management Console (AMC) Usage (0-10)[ , ]","Oracle Java SE Advanced Management Console (AMC) Satisfaction (0-10)[ , ]","Java SE Usage Logging in AMC Usage (0-10)[ , ]","Java SE Usage Logging in AMC Satisfaction (0-10)[ , ]","The Java SE Deployment Rule Set Usage (0-10)[ , ]","The Java SE Deployment Rule Set Satisfaction (0-10)[ , ]","Java SE MSI Customizations Tools Usage (0-10)[ , ]","Java SE MSI Customizations Tools Satisfaction (0-10)[ , ]","Java SE Flight Recorder Usage (0-10)[ , ]","Java SE Flight Recorder Satisfaction (0-10)[ , ]","On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription. Oracle websites","On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription. Oracle Sales professionals","On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription. Oracle marketing materials","On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely important”, please rate the importance of the information channels that increased your awareness of Oracle Java SE Subscription. Blogs","On a scale from 0 to 10 with 0 = “Not at all important” and 10 = “Extremely im

### For dummy0_list vars: <br> &emsp; - # Already one-hot-encoded, just convert NA to 0:

In [397]:
mod_df[dummy0_list].describe(include='all')

,"From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better tools for maintenance and patching]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better tools for automated updates and upgrades ]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [More cost-effective management and monitoring of Java applications]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Greater flexibility]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Familiarity with platform and development tools]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Easier to use]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Access to open source tools]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better support provided by vendor, including legacy Java support]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Greater application security]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Access to skilled development resources]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Lower risk of regulatory non-compliance]","From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Lower risk of application downtime]"
count,42.0,40.0,72.0,80.0,47.0,102.0,91.0,32.0,64.0,50.0,20.0,42.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [398]:
temp = mod_df[dummy0_list].fillna(0)
mod_df.update(temp)

### Convert column names to short_names 

In [399]:
# Create new row with "long_names" for reference
mod_df.loc[-1] = mod_df.columns
mod_df.index = mod_df.index + 1
mod_df= mod_df.sort_index()

# rename columns to short_name
mod_df.columns = short_names

In [400]:
mod_df.shape

(229, 162)

In [401]:
mod_df.head()

,Response_ID,IP_Address,Timestamp,Duplicate,Time_Complete_s,Seq._Number,External_Reference,Custom_Variable_1,Custom_Variable_2,Custom_Variable_3,Custom_Variable_4,Custom_Variable_5,Respondent_Email,Email_List,Country_Code,Region,Survey_Pre_Qual,Job_Title,Job_Title_Other,Company_Industry,Company_Industry_Other],Company_Revenue,OracleJavaSESubscription_Org_Duration,Java_App_Org_Deploy_2020,OracleJavaSESubscription_Ind_Role,OracleJavaSESubscription_Ind_Role_Other,Survey_Consent,Government_Org_Ind_Employee,Gift_Card_Confirm,Company_Name,Thank_You,Title,Country,State,App_Dev_Outsource,Java_Dev_Org,Java_Dev_Dist_Org,Java_Dev_Dist_JVM,Java_Dev_Dist_Python,Java_Dev_Dist_JavaScript,Platform_Dev_Org_Primary,Platform_Dev_Org_Comment,Platform_Dev_Org_LRHEL_P,Platform_Dev_Org_LOracle_P,Platform_Dev_Org_LUbuntu_P,Platform_Dev_Org_LSUSE_P,Platform_Dev_Org_LinuxOther_P,Platform_Dev_Org_MacOS_P,Platform_Dev_Org_Solaris_P,Platform_Dev_Org_Windows_P,Platform_Dev_Org_LinuxOther_Comment,Use_Org_EclipseIDE,Sat_Org_EclipseIDE,Use_Org_IntelliJIDE,Sat_Org_IntelliJIDE,Use_Org_JDeveloper,Sat_Org_JDeveloper,Use_Org_NetBeansIDE,Sat_Org_NetBeansIDE,Use_Org_RationalAppDev,Sat_Org_RationalAppDev,Use_Org_VisualStudio,Sat_Org_VisualStudio,Use_Org_Other,Sat_Org_Other,Use_Sat_Org_Other_Comment,Use_Org_JavaSEAMC,Sat_Org_JavaSEAMC,Use_Org_JavaSEAMC_UsageLog,Sat_Org_JavaSEAMC_UsageLog,Use_Org_JavaSEDepRuleSet,Sat_Org_JavaSEDepRuleSet,Use_Org_JavaSEMSICustTools,Sat_Org_JavaSEMSICustTools,Use_Org_JavaSEFlightRec,Sat_Org_JavaSEFlightRec,JavaSESub_PrePurchasePeriod,Information_Channel_Imp_OWebsites,Information_Channel_Imp_OSalesPros,Information_Channel_Imp_OMarketing,Information_Channel_Imp_Blogs,Information_Channel_Imp_ExtConsultants,Information_Channel_Imp_SocialMedia,Information_Channel_Imp_UserComm,Information_Channel_Imp_IndustryPubs,Information_Channel_Imp_PeerRec,Information_Channel_Imp_Other,Information_Channel_Imp_Other_Comment,OJavaSESubscription_Elem_Imp_Org_ContinuedInvest,OJavaSESubscription_Elem_Sat_Org_ContinuedInvest,OJavaSESubscription_Elem_Imp_Org_PatchesOlderV,OJavaSESubscription_Elem_Sat_Org_PatchesOlderV,OJavaSESubscription_Elem_Imp_Org_JavaWebStartUpdates,OJavaSESubscription_Elem_Sat_Org_JavaWebStartUpdates,OJavaSESubscription_Elem_Imp_Org_DesktopManageUpdates,OJavaSESubscription_Elem_Sat_Org_DesktopManageUpdates,OJavaSESubscription_Elem_Imp_Org_AutoUpdateTools,OJavaSESubscription_Elem_Sat_Org_AutoUpdateTools,OJavaSESubscription_Elem_Imp_Org_Monitoring,OJavaSESubscription_Elem_Sat_Org_Monitoring,OJavaSESubscription_Elem_Imp_Org_MyOSupport,OJavaSESubscription_Elem_Sat_Org_MyOSupport,OJavaSESubscription_Elem_Imp_Org_AccessCloud,OJavaSESubscription_Elem_Sat_Org_AccessCloud,OJavaSESubscription_Elem_Imp_Org_FlexLicensing,OJavaSESubscription_Elem_Sat_Org_FlexLicensing,OJavaSESubscription_Elem_Imp_Org_LegacyJava,OJavaSESubscription_Elem_Sat_Org_LegacyJava,OJavaSESubscription_Elem_Imp_Org_Other,OJavaSESubscription_Elem_Sat_Org_Other,OJavaSESubscription_Elem_Imp_Sat_Org_Other_Comment,OJavaSESubscription_Attr_Imp_CommMethods,OJavaSESubscription_Attr_Sat_CommMethods,OJavaSESubscription_Attr_Imp_PhoneSupport,OJavaSESubscription_Attr_Sat_PhoneSupport,OJavaSESubscription_Attr_Imp_SSKnowledge,OJavaSESubscription_Attr_Sat_SSKnowledge,OJavaSESubscription_Attr_Imp_CoNeedsUnd,OJavaSESubscription_Attr_Sat_CoNeedsUnd,OJavaSESubscription_Attr_Imp_TrainingQual,OJavaSESubscription_Attr_Sat_TrainingQual,OJavaSESubscription_Attr_Imp_RepsonseTime,OJavaSESubscription_Attr_Sat_RepsonseTime,OJavaSESubscription_OrgPurch_Imp_StaffExp,OJavaSESubscription_OrgPurch_Imp_PrPerRatio,OJavaSESubscription_OrgPurch_Imp_TotalCost,OJavaSESubscription_OrgPurch_Imp_LargeEcosystem,OJavaSESubscription_OrgPurch_Imp_VendorSupport,OJavaSESubscription_OrgPurch_Imp_InnovationPotential,OJavaSESubscription_OrgPurch_Imp_LowerRisk,OJavaSESubscription_OrgPurch_Imp_RegulatoryCompliance,OJavaSESubscription_OrgPurch_Imp_MarketTrends,OJavaSESubscription_OrgPurch_Imp_Competition,OJavaSESubscription_OrgPurch_I

### For catg_list vars: <br> &emsp; - convert to categorical & get_dummies()

In [402]:
catg_ind = [raw_df.columns.get_loc(c) for c in catg_list if c in raw_df]
pd.DataFrame({'short':[short_names[x] for x in catg_ind],
              'long':catg_list})

,short,long
0,Country_Code,Country Code
1,Company_Industry,Which one of the below best describes your company’s industry grouping?
2,Job_Title,What is your job title?
3,Company_Revenue,What is the approximate annual revenue of your company?
4,OracleJavaSESubscription_Org_Duration,How long has your organization been using Oracle Java SE Subscription for managing your Java applications?
5,Java_App_Org_Deploy_2020,How many Java applications does your organization plan to build or deploy in 2020?
6,OracleJavaSESubscription_Ind_Role,Which of the following statements most closely describes your role in the selection and use of Oracle Java SE Subscription? (Select one)
7,Government_Org_Ind_Employee,"Are you an employee or official of a government organization (e.g., federal province, state, locality, public higher education institution, etc.), a government owned or controlled entity (e.g., government-owned utilities, etc.), a public international organization, or a political party or candidate for political office?"
8,App_Dev_Outsource,Do you outsource a significant portion of your organization’s application development function?
9,Platform_Dev_Org_Primary,Which platform does your organization primarily develop for?


In [403]:
temp = mod_df[[short_names[x] for x in catg_ind]].apply(lambda x: x.astype('category'))
temp = pd.get_dummies(temp)
temp = temp[['Country_Code_AU', 'Country_Code_CA', 'Country_Code_CZ','Country_Code_DE', 'Country_Code_DK', 
             'Country_Code_GB', 'Country_Code_IN', 'Country_Code_IT', 'Country_Code_NL', 'Country_Code_QA', 
             'Country_Code_US', 'Country_Code_VN', 'Company_Industry_2', 'Company_Industry_5', 'Company_Industry_6', 
             'Company_Industry_7', 'Company_Industry_8', 'Company_Industry_9', 'Company_Industry_10', 
             'Company_Industry_11', 'Company_Industry_12', 'Company_Industry_14', 'Company_Industry_15', 
             'Company_Industry_16', 'Company_Industry_17', 'Company_Industry_18', 'Company_Industry_19', 
             'Company_Industry_20', 'Company_Industry_21', 'Company_Industry_22', 'Company_Industry_23', 
             'Company_Industry_25', 'Job_Title_1', 'Job_Title_2', 'Job_Title_3', 'Job_Title_4', 'Job_Title_5', 
             'Job_Title_8', 'Company_Revenue_1', 'Company_Revenue_2', 'Company_Revenue_3', 'Company_Revenue_4', 
             'OracleJavaSESubscription_Org_Duration_1', 'OracleJavaSESubscription_Org_Duration_2', 
             'OracleJavaSESubscription_Org_Duration_3', 'OracleJavaSESubscription_Org_Duration_4', 
             'Java_App_Org_Deploy_2020_1', 'Java_App_Org_Deploy_2020_2', 'Java_App_Org_Deploy_2020_3', 
             'Java_App_Org_Deploy_2020_4', 'Java_App_Org_Deploy_2020_5',  'OracleJavaSESubscription_Ind_Role_1', 
             'OracleJavaSESubscription_Ind_Role_2', 'OracleJavaSESubscription_Ind_Role_3', 
             'OracleJavaSESubscription_Ind_Role_4', 'OracleJavaSESubscription_Ind_Role_5', 
             'OracleJavaSESubscription_Ind_Role_6', 'Government_Org_Ind_Employee_1', 'Government_Org_Ind_Employee_2',
             'App_Dev_Outsource_1', 'App_Dev_Outsource_2', 'Platform_Dev_Org_Primary_1', 'Platform_Dev_Org_Primary_2',
             'Platform_Dev_Org_Primary_3', 'Platform_Dev_Org_Primary_4', 'Platform_Dev_Org_Primary_5', 
             'Platform_Dev_Org_Primary_6', 'Platform_Dev_Org_Primary_8', 'JavaSESub_PrePurchasePeriod_1', 
             'JavaSESub_PrePurchasePeriod_2', 'JavaSESub_PrePurchasePeriod_3', 'JavaSESub_PrePurchasePeriod_4', 
             'JavaSESub_PrePurchasePeriod_5']]
temp.head(20)

,Country_Code_AU,Country_Code_CA,Country_Code_CZ,Country_Code_DE,Country_Code_DK,Country_Code_GB,Country_Code_IN,Country_Code_IT,Country_Code_NL,Country_Code_QA,Country_Code_US,Country_Code_VN,Company_Industry_2,Company_Industry_5,Company_Industry_6,Company_Industry_7,Company_Industry_8,Company_Industry_9,Company_Industry_10,Company_Industry_11,Company_Industry_12,Company_Industry_14,Company_Industry_15,Company_Industry_16,Company_Industry_17,Company_Industry_18,Company_Industry_19,Company_Industry_20,Company_Industry_21,Company_Industry_22,Company_Industry_23,Company_Industry_25,Job_Title_1,Job_Title_2,Job_Title_3,Job_Title_4,Job_Title_5,Job_Title_8,Company_Revenue_1,Company_Revenue_2,Company_Revenue_3,Company_Revenue_4,OracleJavaSESubscription_Org_Duration_1,OracleJavaSESubscription_Org_Duration_2,OracleJavaSESubscription_Org_Duration_3,OracleJavaSESubscription_Org_Duration_4,Java_App_Org_Deploy_2020_1,Java_App_Org_Deploy_2020_2,Java_App_Org_Deploy_2020_3,Java_App_Org_Deploy_2020_4,Java_App_Org_Deploy_2020_5,OracleJavaSESubscription_Ind_Role_1,OracleJavaSESubscription_Ind_Role_2,OracleJavaSESubscription_Ind_Role_3,OracleJavaSESubscription_Ind_Role_4,OracleJavaSESubscription_Ind_Role_5,OracleJavaSESubscription_Ind_Role_6,Government_Org_Ind_Employee_1,Government_Org_Ind_Employee_2,App_Dev_Outsource_1,App_Dev_Outsource_2,Platform_Dev_Org_Primary_1,Platform_Dev_Org_Primary_2,Platform_Dev_Org_Primary_3,Platform_Dev_Org_Primary_4,Platform_Dev_Org_Primary_5,Platform_Dev_Org_Primary_6,Platform_Dev_Org_Primary_8,JavaSESub_PrePurchasePeriod_1,JavaSESub_PrePurchasePeriod_2,JavaSESub_PrePurchasePeriod_3,JavaSESub_PrePurchasePeriod_4,JavaSESub_PrePurchasePeriod_5
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0
6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1
7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1


In [404]:
catg_dummy_list = temp.columns
catg_dummy_list

Index(['Country_Code_AU', 'Country_Code_CA', 'Country_Code_CZ', 'Country_Code_DE', 'Country_Code_DK', 'Country_Code_GB', 'Country_Code_IN', 'Country_Code_IT', 'Country_Code_NL', 'Country_Code_QA', 'Country_Code_US', 'Country_Code_VN', 'Company_Industry_2', 'Company_Industry_5', 'Company_Industry_6', 'Company_Industry_7', 'Company_Industry_8', 'Company_Industry_9', 'Company_Industry_10', 'Company_Industry_11', 'Company_Industry_12', 'Company_Industry_14', 'Company_Industry_15', 'Company_Industry_16', 'Company_Industry_17', 'Company_Industry_18', 'Company_Industry_19', 'Company_Industry_20', 'Company_Industry_21', 'Company_Industry_22', 'Company_Industry_23', 'Company_Industry_25', 'Job_Title_1', 'Job_Title_2', 'Job_Title_3', 'Job_Title_4', 'Job_Title_5', 'Job_Title_8', 'Company_Revenue_1', 'Company_Revenue_2', 'Company_Revenue_3', 'Company_Revenue_4', 'OracleJavaSESubscription_Org_Duration_1', 'OracleJavaSESubscription_Org_Duration_2', 'OracleJavaSESubscription_Org_Duration_3',
       '

In [405]:
mod_df = mod_df.join(temp)

In [406]:
mod_df = mod_df.iloc[1:]

In [407]:
mod_df.shape

(228, 235)

In [408]:
mod_df = mod_df.apply(pd.to_numeric, errors='coerce')

In [409]:
mod_df.to_csv('../data/output/mod_df.csv', index=False)  

In [410]:
with open('../data/output/catg_dummy_list.csv', 'w', newline="") as f:
    w = csv.writer(f, delimiter = ',')
    w.writerows([x.split(',') for x in catg_dummy_list])

In [411]:
with open('../data/output/catg_dummy_list.csv', 'w', newline="") as f:
    w = csv.writer(f, delimiter = ',')
    w.writerows([x.split(',') for x in catg_dummy_list])

In [412]:
# lines = open('../data/output/catg_dummy_list.csv').read().splitlines()

In [413]:
# cont_list 
# catg_list
# dummy0_list 
# exlc_list 

In [414]:
# cont_list 
cont_ind = [raw_df.columns.get_loc(c) for c in cont_list if c in raw_df]
cont_dummy_list = [short_names[x] for x in cont_ind]
pd.DataFrame({'short':cont_dummy_list,
              'long':cont_list}).head()

,short,long
0,Java_Dev_Org,Approximately how many Java developers are in your organization?
1,Java_Dev_Dist_Org,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on Java
2,Java_Dev_Dist_JVM,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on JVM languages besides Java
3,Java_Dev_Dist_Python,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on Python
4,Java_Dev_Dist_JavaScript,What is the approximate distribution of developers primarily developing on the following development platforms (in percentage)? Note: total %'s can be >100 since many orgs use multiple platforms % developing on JavaScript


In [415]:
with open('../data/output/cont_dummy_list.csv', 'w', newline="") as f:
    w = csv.writer(f, delimiter = ',')
    w.writerows([x.split(',') for x in cont_dummy_list])

In [416]:
# dummy0_list 
dummy0_ind = [raw_df.columns.get_loc(c) for c in dummy0_list if c in raw_df]
dummy0_dummy_list = [short_names[x] for x in dummy0_ind]
pd.DataFrame({'short':dummy0_dummy_list,
              'long':dummy0_list}).head()

,short,long
0,OJavaSESubscription_Alts_Top3_MaintenanceTools,"From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better tools for maintenance and patching]"
1,OJavaSESubscription_Alts_Top3_UpdatesTools,"From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Better tools for automated updates and upgrades ]"
2,OJavaSESubscription_Alts_Top3_CostManagement,"From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [More cost-effective management and monitoring of Java applications]"
3,OJavaSESubscription_Alts_Top3_Flexibility,"From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Greater flexibility]"
4,OJavaSESubscription_Alts_Top3_Familiarity,"From the list below, please check the top 3 reasons for which you would consider alternatives to Oracle Java SE Subscription. [Familiarity with platform and development tools]"


In [417]:
with open('../data/output/dummy0_dummy_list.csv', 'w', newline="") as f:
    w = csv.writer(f, delimiter = ',')
    w.writerows([x.split(',') for x in dummy0_dummy_list])

In [418]:
# dummy0_list 
exlc_ind = [raw_df.columns.get_loc(c) for c in exlc_list if c in raw_df]
exlc_dummy_list = [short_names[x] for x in exlc_ind]
pd.DataFrame({'short':exlc_dummy_list,
              'long': exlc_list}).head()

,short,long
0,Response_ID,Response ID
1,IP_Address,IP Address
2,Timestamp,Timestamp (mm/dd/yyyy)
3,Duplicate,Duplicate
4,Time_Complete_s,Time Taken to Complete (Seconds)


In [419]:
with open('../data/output/exlc_dummy_list.csv', 'w', newline="") as f:
    w = csv.writer(f, delimiter = ',')
    w.writerows([x.split(',') for x in exlc_dummy_list])

In [420]:
mod_df['Java_Dev_Org'].dtype

dtype('float64')

In [421]:
map_df.to_csv('../data/output/map_df.csv', index=False)  